In [1]:
# Import Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, classification_report, precision_recall_curve, auc, accuracy_score, precision_score, recall_score, f1_score
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import LabelEncoder
import warnings
import shap
import numpy as np
from art.attacks.poisoning import PoisoningAttackBackdoor, FeatureCollisionAttack
from art.estimators.classification import SklearnClassifier
from art.utils import to_categorical

sns.set(style="whitegrid")
warnings.filterwarnings('ignore')

In [2]:
# Load the preprocessed data
data = pd.read_csv('Preprocessed_Data.csv')

# Rename columns to remove special characters
data.rename(columns={
    'Air temperature [K]': 'Air_temperature_K',
    'Process temperature [K]': 'Process_temperature_K',
    'Rotational speed [rpm]': 'Rotational_speed_rpm',
    'Torque [Nm]': 'Torque_Nm',
    'Tool wear [min]': 'Tool_wear_min'
}, inplace=True)

# Create the 'No failure' column
data['No failure'] = 1 - data['Machine failure']

# Define features and target
X = data[['Type', 'Air_temperature_K', 'Process_temperature_K', 'Rotational_speed_rpm', 'Torque_Nm', 'Tool_wear_min']]
y = data[['No failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']].idxmax(axis=1)

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
set(y_encoded)

{0, 1, 2, 3, 4}

In [3]:
# Initialize stratified split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in sss.split(X, y_encoded):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

# Apply SMOTE to oversample the training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Define all possible target names
all_classes = label_encoder.classes_
set(all_classes)

{'HDF', 'No failure', 'OSF', 'PWF', 'TWF'}

In [4]:
# Define models with the best parameters
models = {
    'Random Forest': RandomForestClassifier(
        max_depth=35, 
        min_samples_split=3, 
        n_estimators=150, 
        random_state=42
    ),
    'XGBoost': XGBClassifier(
        learning_rate=0.4, 
        max_depth=5, 
        n_estimators=400, 
        subsample=1.0, 
        random_state=42
    ),
    'Neural Network': MLPClassifier(
        activation='relu', 
        hidden_layer_sizes=(50, 50), 
        solver='adam', 
        max_iter=350, 
        random_state=42
    )
}

In [5]:
# Define the poisoning function for multiclass classification
def label_flip_poisoning(X_train, y_train, poison_percentage, target_class):
    # Target Class 1: Flipping "no failure" (y_train == 1) to the 6th failure (RNF, y_train == 6)
    if target_class == 1:
        # Find indices where y_train indicates no failure (y_train == 1)
        no_failure_indices = np.where(y_train == 1)[0]
        
        # Determine the number of labels to flip based on the poison percentage
        num_to_flip = int(poison_percentage * len(no_failure_indices))
        
        # Randomly select indices to flip
        flip_indices = np.random.choice(no_failure_indices, size=num_to_flip, replace=False)
        
        # Create poisoned labels by copying y_train
        y_train_poisoned = y_train.copy()
        
        # Flip the selected "no failure" labels (1) to the 6th failure (RNF, y_train == 6)
        y_train_poisoned[flip_indices] = 0
    
    # Target Class 2: Flipping failure types (2 to 6) to "no failure" (y_train == 1)
    else:
        # Find indices where y_train indicates failure (y_train in [2, 3, 4, 5, 6])
        failure_indices = np.where(y_train > 1)[0]
        
        # Determine the number of labels to flip based on the poison percentage
        num_to_flip = int(poison_percentage * len(failure_indices))
        
        # Randomly select indices to flip
        flip_indices = np.random.choice(failure_indices, size=num_to_flip, replace=False)
        
        # Create poisoned labels by copying y_train
        y_train_poisoned = y_train.copy()
        
        # Flip the selected failure labels (2 to 6) to "no failure" (1)
        y_train_poisoned[flip_indices] = 1
    
    return X_train, y_train_poisoned


In [6]:
# Detect suspicious labels using KNN and return the most common neighbors for correction
def identify_suspicious_labels(X_train, y_train, threshold=0.7, n_neighbors=3):
    # Ensure X_train is a NumPy array for KNN compatibility
    if isinstance(X_train, pd.DataFrame):
        X_train = X_train.values
    
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_train)
    
    # Identify mismatches between the predicted labels and the actual labels
    mismatches = (y_train != y_pred_knn)
    suspicious_idx = np.where(mismatches)[0]
    
    # Initialize a list to store the most common neighbor class for each suspicious point
    most_common_neighbors = []
    
    # Loop through suspicious indices and find the most common neighbor class
    for idx in suspicious_idx:
        neighbor_indices = knn.kneighbors([X_train[idx]], return_distance=False)[0]
        
        # Find the most common class among the neighbors
        neighbor_classes = y_train[neighbor_indices]  # Since y_train is a NumPy array, we use direct indexing
        most_common_class = np.bincount(neighbor_classes).argmax()  # Get the most frequent class
        most_common_neighbors.append(most_common_class)
    
    print(f"Number of suspicious indices: {len(suspicious_idx)}")
    return suspicious_idx, most_common_neighbors


In [7]:
# Correct the suspicious labels by assigning them the most common class among neighbors
def correct_labels_failure_type(X_train, y_train, suspicious_idx, most_common_neighbors):
    corrected_labels = y_train.copy()
    
    # Replace the suspicious labels with the most common class from neighbors
    for i, idx in enumerate(suspicious_idx):
        corrected_labels.iloc[idx] = most_common_neighbors[i]
    
    return corrected_labels
# Define label deletion
def delete_labels(X_train, y_train, suspicious_idx):
    X_train_cleaned = np.delete(X_train, suspicious_idx, axis=0)
    y_train_cleaned = np.delete(y_train, suspicious_idx, axis=0)
    return X_train_cleaned, y_train_cleaned

In [8]:
# Define the poisoning percentages to test
poison_percentages = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
target_classes = [1, 2]  # 0 for no failure, 1 for failure

# Initialize dictionaries to hold DataFrames for each metric, model, target class, and intervention type
accuracy_results = {'Model': [], 'Target': [], 'Intervention': []}
precision_results = {'Model': [], 'Target': [], 'Intervention': []}
recall_results = {'Model': [], 'Target': [], 'Intervention': []}
f1_results = {'Model': [], 'Target': [], 'Intervention': []}
fdr_results = {'Model': [], 'Target': [], 'Intervention': []}  # For Failure Detection Rate

# Add columns for each poisoning percentage in the results dictionaries
for poison_percentage in poison_percentages:
    accuracy_results[f'{int(poison_percentage * 100)}%'] = []
    precision_results[f'{int(poison_percentage * 100)}%'] = []
    recall_results[f'{int(poison_percentage * 100)}%'] = []
    f1_results[f'{int(poison_percentage * 100)}%'] = []
    fdr_results[f'{int(poison_percentage * 100)}%'] = []  # FDR results

# Add a column to store the mean of each metric across the poisoning percentages
accuracy_results['Mean'] = []
precision_results['Mean'] = []
recall_results['Mean'] = []
f1_results['Mean'] = []
fdr_results['Mean'] = []  # FDR Mean


In [9]:
# Define the interventions
interventions = ['No Intervention', 'Correction', 'Deletion']

# Loop through each model
for name, model in models.items():
    print(f'Model: {name}')
    
    # Loop through each target class (0 and 1)
    for target_class in target_classes:
        print(f'Target Class: {target_class}')
        
        # Loop through each intervention type (No Intervention, Correction, Deletion)
        for intervention in interventions:
            print(f'Intervention: {intervention}')
            
            # Store the model name, target class, and intervention type in the results dictionaries
            accuracy_results['Model'].append(name)
            precision_results['Model'].append(name)
            recall_results['Model'].append(name)
            f1_results['Model'].append(name)
            fdr_results['Model'].append(name)

            accuracy_results['Target'].append(target_class)
            precision_results['Target'].append(target_class)
            recall_results['Target'].append(target_class)
            f1_results['Target'].append(target_class)
            fdr_results['Target'].append(target_class)

            accuracy_results['Intervention'].append(intervention)
            precision_results['Intervention'].append(intervention)
            recall_results['Intervention'].append(intervention)
            f1_results['Intervention'].append(intervention)
            fdr_results['Intervention'].append(intervention)
            
            # Store a list to calculate the mean values later
            acc_values = []
            prec_values = []
            rec_values = []
            f1_values = []
            fdr_values = []
            
            # Loop through each poisoning percentage (including 0% for clean data)
            for poison_percentage in poison_percentages:
                print(f'Poison percentage: {poison_percentage}')

                # Poison the training data
                X_train_poisoned, y_train_poisoned = label_flip_poisoning(X_train_res, y_train_res, poison_percentage, target_class)
                
                # Apply label correction or deletion if needed
                if intervention == 'Correction':
                    suspicious_indices, most_common_neighbors = identify_suspicious_labels(X_train_poisoned, y_train_poisoned)
                    y_train_poisoned[suspicious_indices] = most_common_neighbors  # Correct labels based on neighbors
                elif intervention == 'Deletion':
                    suspicious_indices, _ = identify_suspicious_labels(X_train_poisoned, y_train_poisoned)
                    X_train_poisoned, y_train_poisoned = delete_labels(X_train_poisoned, y_train_poisoned, suspicious_indices)

                # Train the model on the (possibly corrected or cleaned) poisoned data
                model.fit(X_train_poisoned, y_train_poisoned)
                
                # Make predictions on the clean test set
                y_pred = model.predict(X_test)
                
                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, average='weighted')
                recall = recall_score(y_test, y_pred, average='weighted')
                f1 = f1_score(y_test, y_pred, average='weighted')
                
                # Calculate Failure Detection Rate (FDR) for class 2 (failure)
                if target_class == 2:
                    true_failure = (y_test == 2)
                    true_positive_failures = ((y_pred == 2) & (y_test == 2)).sum()
                    fdr = true_positive_failures / true_failure.sum() if true_failure.sum() > 0 else 0
                else:
                    fdr = None  # FDR is only relevant for the failure class
                
                # Store the metrics based on the poison percentage for the current intervention and target class
                accuracy_results[f'{int(poison_percentage * 100)}%'].append(accuracy)
                precision_results[f'{int(poison_percentage * 100)}%'].append(precision)
                recall_results[f'{int(poison_percentage * 100)}%'].append(recall)
                f1_results[f'{int(poison_percentage * 100)}%'].append(f1)
                fdr_results[f'{int(poison_percentage * 100)}%'].append(fdr)

                # Store the metrics for mean calculation
                acc_values.append(accuracy)
                prec_values.append(precision)
                rec_values.append(recall)
                f1_values.append(f1)
                if fdr is not None:
                    fdr_values.append(fdr)

            # Calculate the mean of each metric across all poisoning percentages
            accuracy_results['Mean'].append(sum(acc_values) / len(acc_values))
            precision_results['Mean'].append(sum(prec_values) / len(prec_values))
            recall_results['Mean'].append(sum(rec_values) / len(rec_values))
            f1_results['Mean'].append(sum(f1_values) / len(f1_values))
            if fdr_values:
                fdr_results['Mean'].append(sum(fdr_values) / len(fdr_values))
            else:
                fdr_results['Mean'].append(None)

# Convert the results dictionaries to DataFrames
accuracy_df = pd.DataFrame(accuracy_results)
precision_df = pd.DataFrame(precision_results)
recall_df = pd.DataFrame(recall_results)
f1_df = pd.DataFrame(f1_results)
fdr_df = pd.DataFrame(fdr_results)  # FDR DataFrame

# Display the results
print("Accuracy Results:")
display(accuracy_df)

print("Precision Results:")
display(precision_df)

print("Recall Results:")
display(recall_df)

print("F1 Score Results:")
display(f1_df)

print("Failure Detection Rate (FDR) Results:")
display(fdr_df)

Model: Random Forest
Target Class: 1
Intervention: No Intervention
Poison percentage: 0
Poison percentage: 0.1
Poison percentage: 0.2
Poison percentage: 0.3
Poison percentage: 0.4
Poison percentage: 0.5
Intervention: Correction
Poison percentage: 0
Number of suspicious indices: 213
Poison percentage: 0.1
Number of suspicious indices: 801
Poison percentage: 0.2
Number of suspicious indices: 1247
Poison percentage: 0.3
Number of suspicious indices: 1587
Poison percentage: 0.4
Number of suspicious indices: 1789
Poison percentage: 0.5
Number of suspicious indices: 1809
Intervention: Deletion
Poison percentage: 0
Number of suspicious indices: 213
Poison percentage: 0.1
Number of suspicious indices: 785
Poison percentage: 0.2
Number of suspicious indices: 1221
Poison percentage: 0.3
Number of suspicious indices: 1559
Poison percentage: 0.4
Number of suspicious indices: 1720
Poison percentage: 0.5
Number of suspicious indices: 1802
Target Class: 2
Intervention: No Intervention
Poison percenta

,Model,Target,Intervention,0%,10%,20%,30%,40%,50%,Mean
0,Random Forest,1,No Intervention,0.969667,0.965333,0.954667,0.910000,0.770000,0.488000,0.842944
1,Random Forest,1,Correction,0.956667,0.952333,0.945000,0.912333,0.782333,0.467667,0.836056
2,Random Forest,1,Deletion,0.961000,0.959000,0.952667,0.919667,0.796333,0.448667,0.839556
3,Random Forest,2,No Intervention,0.969667,0.974667,0.977000,0.978000,0.977667,0.977000,0.975667
4,Random Forest,2,Correction,0.956667,0.960000,0.965667,0.970000,0.972333,0.974000,0.966444
5,Random Forest,2,Deletion,0.961000,0.968000,0.969000,0.971667,0.973667,0.976667,0.970000
6,XGBoost,1,No Intervention,0.979333,0.952000,0.884333,0.802000,0.662000,0.479333,0.793167
7,XGBoost,1,Correction,0.958000,0.951333,0.917667,0.846333,0.696667,0.483667,0.808944
8,XGBoost,1,Deletion,0.970000,0.960667,0.939333,0.868000,0.714667,0.473000,0.820944
9,XGBoost,2,No Intervention,0.979333,0.980333,0.978667,0.978667,0.980667,0.981333,0.979833


Precision Results:


,Model,Target,Intervention,0%,10%,20%,30%,40%,50%,Mean
0,Random Forest,1,No Intervention,0.979279,0.978381,0.977543,0.974910,0.973733,0.974790,0.976439
1,Random Forest,1,Correction,0.978247,0.977658,0.976661,0.973680,0.971906,0.971947,0.975017
2,Random Forest,1,Deletion,0.977740,0.977701,0.977956,0.974448,0.972563,0.970427,0.975139
3,Random Forest,2,No Intervention,0.979279,0.976288,0.976899,0.976646,0.974367,0.971938,0.975903
4,Random Forest,2,Correction,0.978247,0.977950,0.976110,0.976292,0.973862,0.972620,0.975847
5,Random Forest,2,Deletion,0.977740,0.978536,0.976460,0.975661,0.971773,0.975426,0.975933
6,XGBoost,1,No Intervention,0.985434,0.977864,0.976376,0.973470,0.975215,0.972312,0.976779
7,XGBoost,1,Correction,0.979576,0.976646,0.974450,0.973526,0.973174,0.972180,0.974925
8,XGBoost,1,Deletion,0.983198,0.980402,0.977551,0.974460,0.975317,0.974920,0.977641
9,XGBoost,2,No Intervention,0.985434,0.983526,0.981088,0.979166,0.979335,0.977264,0.980969


Recall Results:


,Model,Target,Intervention,0%,10%,20%,30%,40%,50%,Mean
0,Random Forest,1,No Intervention,0.969667,0.965333,0.954667,0.910000,0.770000,0.488000,0.842944
1,Random Forest,1,Correction,0.956667,0.952333,0.945000,0.912333,0.782333,0.467667,0.836056
2,Random Forest,1,Deletion,0.961000,0.959000,0.952667,0.919667,0.796333,0.448667,0.839556
3,Random Forest,2,No Intervention,0.969667,0.974667,0.977000,0.978000,0.977667,0.977000,0.975667
4,Random Forest,2,Correction,0.956667,0.960000,0.965667,0.970000,0.972333,0.974000,0.966444
5,Random Forest,2,Deletion,0.961000,0.968000,0.969000,0.971667,0.973667,0.976667,0.970000
6,XGBoost,1,No Intervention,0.979333,0.952000,0.884333,0.802000,0.662000,0.479333,0.793167
7,XGBoost,1,Correction,0.958000,0.951333,0.917667,0.846333,0.696667,0.483667,0.808944
8,XGBoost,1,Deletion,0.970000,0.960667,0.939333,0.868000,0.714667,0.473000,0.820944
9,XGBoost,2,No Intervention,0.979333,0.980333,0.978667,0.978667,0.980667,0.981333,0.979833


F1 Score Results:


,Model,Target,Intervention,0%,10%,20%,30%,40%,50%,Mean
0,Random Forest,1,No Intervention,0.974029,0.971069,0.963998,0.936686,0.852114,0.636554,0.889075
1,Random Forest,1,Correction,0.965962,0.963053,0.957990,0.937342,0.858905,0.615899,0.883192
2,Random Forest,1,Deletion,0.968320,0.967112,0.963293,0.942006,0.867797,0.597262,0.884299
3,Random Forest,2,No Intervention,0.974029,0.975263,0.976414,0.976050,0.973130,0.970987,0.974312
4,Random Forest,2,Correction,0.965962,0.967995,0.970342,0.972559,0.971878,0.969942,0.969780
5,Random Forest,2,Deletion,0.968320,0.972778,0.972391,0.973157,0.971340,0.971905,0.971648
6,XGBoost,1,No Intervention,0.982062,0.962225,0.922770,0.872985,0.779660,0.628631,0.858056
7,XGBoost,1,Correction,0.967254,0.961956,0.940801,0.899311,0.802590,0.632245,0.867359
8,XGBoost,1,Deletion,0.975781,0.968837,0.954686,0.912230,0.815883,0.622417,0.874972
9,XGBoost,2,No Intervention,0.982062,0.981813,0.979709,0.978649,0.978741,0.976500,0.979579


Failure Detection Rate (FDR) Results:


,Model,Target,Intervention,0%,10%,20%,30%,40%,50%,Mean
0,Random Forest,1,No Intervention,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Random Forest,1,Correction,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Random Forest,1,Deletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Random Forest,2,No Intervention,0.869565,0.521739,0.478261,0.391304,0.217391,0.173913,0.442029
4,Random Forest,2,Correction,0.826087,0.826087,0.652174,0.608696,0.347826,0.217391,0.579710
5,Random Forest,2,Deletion,0.869565,0.869565,0.652174,0.565217,0.347826,0.173913,0.579710
6,XGBoost,1,No Intervention,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,XGBoost,1,Correction,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,XGBoost,1,Deletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,XGBoost,2,No Intervention,0.913043,0.695652,0.608696,0.608696,0.391304,0.260870,0.579710


In [18]:
# Function to plot a metric for a model across poisoning percentages, grouped by intervention type
def plot_metric_separate(metric_df, metric_name, model_name, target_class):
    plt.figure(figsize=(10, 6))

    # Filter the DataFrame for the selected target class and model
    filtered_df = metric_df[(metric_df['Target'] == target_class) & (metric_df['Model'] == model_name)]

    # Iterate through each intervention and plot its metric across poisoning percentages (excluding the mean column)
    for intervention in filtered_df['Intervention'].unique():
        # Filter by intervention type
        intervention_df = filtered_df[filtered_df['Intervention'] == intervention]

        # Get the values as numpy array, stripping the '%' symbol and converting to integers
        # Exclude the 'Mean' column from the x_values
        x_values = [int(col.strip('%')) for col in intervention_df.columns[3:-1]]  # Skip 'Model', 'Target', 'Intervention', and 'Mean'
        y_values = intervention_df.iloc[0, 3:-1].values  # Get the corresponding metric values

        # Plot the line for each intervention
        plt.plot(x_values, y_values, label=intervention, marker='o')

    # Add labels and title
    plt.title(f'{metric_name} Across Poisoning Percentages for {model_name} (Target Class: {target_class})', fontsize=16)
    plt.xlabel('Poisoning Percentage (%)', fontsize=12)
    plt.ylabel(metric_name, fontsize=12)

    # Display legend
    plt.legend(title='Intervention', bbox_to_anchor=(1.05, 1), loc='upper left')

    # Show plot
    plt.tight_layout()
    plt.show()

# Define a function that will update the plot based on user input
def interactive_plot_separate(metric_name, model_name, target_class):
    if metric_name == 'Accuracy':
        plot_metric_separate(accuracy_df, "Accuracy", model_name, target_class)
    elif metric_name == 'Precision':
        plot_metric_separate(precision_df, "Precision", model_name, target_class)
    elif metric_name == 'Recall':
        plot_metric_separate(recall_df, "Recall", model_name, target_class)
    elif metric_name == 'F1 Score':
        plot_metric_separate(f1_df, "F1 Score", model_name, target_class)
    elif metric_name == 'Failure Detection Rate':
        plot_metric_separate(fdr_df, "Failure Detection Rate", model_name, target_class)  # FDR plot

# Create a dropdown menu for selecting the metric (including FDR)
metric_dropdown = widgets.Dropdown(
    options=['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Failure Detection Rate'],
    value='Accuracy',
    description='Metric:',
    disabled=False,
)

# Create a dropdown menu for selecting the model (Random Forest, XGBoost, Neural Network)
model_dropdown = widgets.Dropdown(
    options=['Random Forest', 'XGBoost', 'Neural Network'],
    value='Random Forest',
    description='Model:',
    disabled=False,
)

# Create a dropdown menu for selecting the target class (0 or 1)
target_dropdown = widgets.Dropdown(
    options=[1, 2],
    value=1,
    description='Target Class:',
    disabled=False,
)

# Use the interact function to update the plot dynamically
interact(interactive_plot_separate, metric_name=metric_dropdown, model_name=model_dropdown, target_class=target_dropdown)

interactive(children=(Dropdown(description='Metric:', options=('Accuracy', 'Precision', 'Recall', 'F1 Score', …

<function __main__.interactive_plot_separate(metric_name, model_name, target_class)>